# ADA - Project
## With Spark

## Getting started

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext("local[*]", "ADA")
sc.defaultParallelism

8

In [3]:
data = sc.parallelize(range(100))
data.sum()

4950

If you received 4950 as a result, your spark is working well :) Good job !

## Other imports

In [4]:
import pandas as pd
import random

import pprint
pp = pprint.PrettyPrinter(indent=4)

## Configuration

In [5]:
config = {
    "session":"progfun-002"
}

## Importing and parsing the data

### Importing Event Data

In [6]:
# Reading csv files: Create RDD () with one string entry per line in the file
rdd_problem_events = sc.textFile("data/"+config['session']+"_Problem_Events_with_Info.csv")
rdd_video_events = sc.textFile("data/"+config['session']+"_Video_Events.csv")
rdd_forum_events = sc.textFile("data/"+config['session']+"_Forum_Events.csv")

# Prints the first line (header) along with indices for each table
print("--- PROBLEM_EVENTS ---")
for idx,field in enumerate(rdd_problem_events.first().split(",")): 
    print(idx,field, end=" / ")

print("\n\n--- VIDEO_EVENTS ---")
for idx,field in enumerate(rdd_video_events.first().split(",")): 
    print(idx,field, end=" / ")

print("\n\n--- FORUM_EVENTS ---")
for idx,field in enumerate(rdd_forum_events.first().split(",")): 
    print(idx,field, end=" / ")

--- PROBLEM_EVENTS ---
0 EventID / 1 ForumUserID / 2 MaximumSubmissions / 3 AccountUserID / 4 SubmissionNumber / 5 Grade / 6 TimeStamp / 7 DataPackageID / 8 ProblemID / 9 SoftCloseTime / 10 ProblemType / 11 HardCloseTime / 12 Platform / 13 OpenTime / 14 EventType / 15 Title / 16 SessionUserID / 17 UniqueProblemID / 18 UniqueUserID / 

--- VIDEO_EVENTS ---
0 EventID / 1 ForumUserID / 2 OldTime / 3 AccountUserID / 4 CurrentTime / 5 SeekType / 6 TimeStamp / 7 DataPackageID / 8 UniqueRowID / 9 TableName / 10 VideoID / 11 Platform / 12 NewSpeed / 13 EventSource / 14 EventType / 15 SessionUserID / 16 NewTime / 17 OldSpeed / 

--- FORUM_EVENTS ---
0 EventID / 1 ForumUserID / 2 PostID / 3 AccountUserID / 4 TimeStamp / 5 DataPackageID / 6 UniqueRowID / 7 TableName / 8 Platform / 9 EventSource / 10 PostType / 11 EventType / 12 JoinID / 13 SessionUserID / 

In [7]:
# Use the filter method to remove the first line
rdd_problem_events = rdd_problem_events.filter(lambda x: not x.startswith('EventID'))
rdd_video_events = rdd_video_events.filter(lambda x: not x.startswith('EventID'))
rdd_forum_events = rdd_forum_events.filter(lambda x: not x.startswith('EventID'))

# Prints first record for each table
print(rdd_problem_events.first(),"\n") 
print(rdd_video_events.first(),"\n") 
print(rdd_forum_events.first(),"\n")

3bcd1a54ed6ddb04b4a6fb2906110a01,None,0,None,7,None,1365344171,progfun-002,7,2147483640,Video,2147483640,Coursera,32400,Problem.Check,Lecture 1.2 - Elements of Programming (14:25),c4d4e5fcd2feba9f3234ee8d852dc7b22fbc07e4,f322944718b2ee0e53292118111533c7,21f13b3f6b50a83343b57d2f1d07dbdf 

db75adce6b87e7ab79242ea0af4b82d4,None,154.696,None,154.697,None,1372391638,progfun-002,00000078c0f0685cc50a25a8d5734a88,Video_Events,33,coursera,1.0,None,Video.Play,ef64fb7b096008f7eaf8441684afdf99af9af54a,None,1.0 

f3fdb52859b2511308aee554a573194e,None,17,4108315,1376254235,progfun-002,000006c12322ca29c7013dac42ef1a6a,Forum_Events,coursera,None,Thread,Forum.Thread.View,03b1fa287de5ef57d9c8482195b5167f,None 



In [8]:
# Function to parse the string entries from previous dataset
def parse_problems(x):
    data = x.split(',')
    return {
        'Grade':float(data[5]),
        'TimeStamp':int(data[6]),
        'Date':pd.to_datetime(int(data[6]),unit='s'),
        'ProblemID':int(data[8]),
        'ProblemType':data[10],
        'EventType':data[14].split('.')[0],
        'EventSubType':data[14].split('.')[1],
        'Title':data[15],
        'SessionUserID':data[16]
    }

def parse_videos(x):
    data = x.split(',')
    return {
        'TimeStamp':int(data[6]),
        'Date':pd.to_datetime(int(data[6]),unit='s'),
        'VideoID':int(data[10]),
        'EventType':data[14].split('.')[0],
        'EventSubType':data[14].split('.')[1],
        'SessionUserID':data[15]
    }

def parse_forums(x):
    data = x.split(',')
    return {
        'AccountUserID':data[3],
        'TimeStamp':int(data[4]),
        'Date':pd.to_datetime(int(data[4]),unit='s'),
        'EventType':data[11].split('.')[0],
        'EventSubType':data[11].split('.')[1]
    }

In [9]:
# Function to filter the string entries from previous dataset
def filter_problems(x):
    data = x.split(',')
    return (
        # Some problem event are quiz inside video, we decide to not consider these
        (data[10]=='Assignment')
        # We remove IDs of assignments that have been used by the teaching staff for testing the platform
        and (not data[8] in ['1','2','3','4','5'])
        # Discard assignments with no grade
        and (not data[5] in ['None'])
    )

def filter_videos(x):
    data = x.split(',')
    return (
        # We remove video that do not belong to the MOOC lectures or ar just Setup videos
        (data[10] not in ['9','12','11','10','13','2','29','25','21','27','23'])
        # Keeps only "Play" EnventSubStype for videos (makes difference between opening the page or starting the video)
        and (data[14].split('.')[1] in ['Play','Load'])
    )

def filter_forums(x):
    data = x.split(',')
    return (
        True
    )

In [10]:
# Use the map method to have more workable data
rdd_problem_events_parsed = rdd_problem_events.filter(filter_problems).map(parse_problems)
rdd_video_events_parsed = rdd_video_events.filter(filter_videos).map(parse_videos)
rdd_forum_events_parsed = rdd_forum_events.filter(filter_forums).map(parse_forums)

# Prints the count of elements along with the first element of each table
print(rdd_problem_events_parsed.count())
print(rdd_problem_events_parsed.first(),"\n") 
print(rdd_video_events_parsed.count()) 
print(rdd_video_events_parsed.first(),"\n") 
print(rdd_forum_events_parsed.count()) 
print(rdd_forum_events_parsed.first(),"\n") 

90325
{'SessionUserID': 'd8f79efa32a560b8a46ea2b12d9bed97c9e39b4b', 'ProblemID': 6, 'TimeStamp': 1366220878, 'ProblemType': 'Assignment', 'EventSubType': 'Check', 'Date': Timestamp('2013-04-17 17:47:58'), 'Grade': 9.32999992371, 'Title': 'Functional Sets / Functional Sets', 'EventType': 'Problem'} 

1168226
{'SessionUserID': 'ef64fb7b096008f7eaf8441684afdf99af9af54a', 'VideoID': 33, 'EventSubType': 'Play', 'Date': Timestamp('2013-06-28 03:53:58'), 'TimeStamp': 1372391638, 'EventType': 'Video'} 

297650
{'EventSubType': 'Thread', 'Date': Timestamp('2013-08-11 20:50:35'), 'TimeStamp': 1376254235, 'AccountUserID': '4108315', 'EventType': 'Forum'} 



In [11]:
# Handles issue with the forum events table having 'AccountUserID' instead of 'SessionUserID'
# Using the table progfun-002_User_Hash_Mapping
rdd_user_mapping = sc.textFile("data/"+config['session']+"_User_Hash_Mapping.csv")
print(rdd_user_mapping.take(1)[0],"\n")

def f(x):
    x[1][1]['SessionUserID']=x[1][0] 
    return x[1][1]

rdd_forum_events_parsed = (rdd_user_mapping
    # removes header
    .filter(lambda x: not x.startswith("ForumUserID"))
    # maps to have the format (Key=AccountUserID,Value=SessionUserID)
    .map(lambda x:(x.split(",")[1],x.split(",")[6]))
    # join with rdd_forum_event to get format (map to have the (Key=AccountUserID,Value=(SessionUserID,EventObject)) format)
    .join(rdd_forum_events_parsed
        # map to have the (Key=AccountUserID,Value=EventObject) format
        .map(lambda x: (x['AccountUserID'],x))
    )
    # Use the function f to update EventObject with the joined SessionUserID
    .map(f)
)

print(rdd_forum_events_parsed.count())
print(rdd_forum_events_parsed.take(1)[0])

ForumUserID,AccountUserID,DataPackageID,UniqueRowID,TableName,Platform,SessionUserID 

297650
{'SessionUserID': '16e564ed1e9b7104acde8f075c26a1872695fb5e', 'EventSubType': 'Thread', 'Date': Timestamp('2013-05-09 20:22:34'), 'TimeStamp': 1368130954, 'AccountUserID': '2009208', 'EventType': 'Forum'}


In [12]:
# Concatenantes all three table into one big table
rdd_events = (rdd_problem_events_parsed
    .union(rdd_video_events_parsed)
    .union(rdd_forum_events_parsed)
)
rdd_events.persist()
print(rdd_events.count())
print(rdd_events.take(1)[0])

1556201
{'SessionUserID': 'd8f79efa32a560b8a46ea2b12d9bed97c9e39b4b', 'Title': 'Functional Sets / Functional Sets', 'Date': Timestamp('2013-04-17 17:47:58'), 'EventSubType': 'Check', 'ProblemType': 'Assignment', 'Grade': 9.32999992371, 'EventType': 'Problem', 'TimeStamp': 1366220878, 'ProblemID': 6}


### Importing meta data
The goal is to have the table linking VideoIDs to corresponding ProblemIDs

In [13]:
assignments = sorted((rdd_problem_events_parsed
    .map(lambda x: (x['ProblemID'],x['Title']))
    .distinct()
    .collect()
),key=(lambda x: x[0]))

In [14]:
rdd_video_info = sc.textFile("data/"+config['session']+"_Video_Info.csv")
videos = sorted((rdd_video_info
    .map(lambda x: x.split(','))
    .filter(lambda x: not x[1] in ['None', 'OpenTime'] )
    .map(lambda x: (int(x[7]),x[2]))
    .collect()
),key=(lambda x: x[0]))

In [15]:
# Made by hand... so painful
LECTURES_PER_PROBLEM = {
    7: [3,4,5,6,7,8,33], # Lecture 1
    6: [35,37,39,41,43,47,49], # Lecture 2
    12: [51,53,75], # Lecture 3
    14: [71,81,85,79,87,77], # Lecture 4
    17: [109,105,115,107,103,113,111], # Lecture 6
    20: [123,117,125,121,127,119] # Lecture 7
}
LECTURES_PER_PROBLEM

{6: [35, 37, 39, 41, 43, 47, 49],
 7: [3, 4, 5, 6, 7, 8, 33],
 12: [51, 53, 75],
 14: [71, 81, 85, 79, 87, 77],
 17: [109, 105, 115, 107, 103, 113, 111],
 20: [123, 117, 125, 121, 127, 119]}

In [16]:
PROBLEM_PER_LECTURE = {}
for pb in LECTURES_PER_PROBLEM.keys():
    for lc in LECTURES_PER_PROBLEM[pb]:
        PROBLEM_PER_LECTURE[lc]=pb
print(PROBLEM_PER_LECTURE)

{51: 12, 3: 7, 4: 7, 49: 6, 6: 7, 7: 7, 8: 7, 119: 20, 77: 14, 79: 14, 43: 6, 75: 12, 85: 14, 87: 14, 39: 6, 71: 14, 111: 17, 81: 14, 5: 7, 33: 7, 35: 6, 37: 6, 103: 17, 105: 17, 107: 17, 109: 17, 47: 6, 113: 17, 53: 12, 115: 17, 117: 20, 41: 6, 121: 20, 123: 20, 125: 20, 127: 20}


## Working the data

### Grouping & Sorting

In [17]:
# uses the function groupByKey on our events with the key 'SessionStudentID'
rdd_events_by_students = (rdd_events
    .map(lambda x: (x['SessionUserID'],x))
    # Groups the list of event by student
    .groupByKey()
    # Sorts each student sequence by the timestamp
    .map(lambda x: (x[0],sorted(x[1], key=(lambda event: event['TimeStamp']))))
)
rdd_events_by_students.persist()
print("Number of students: %d" % rdd_events_by_students.count()) 

Number of students: 24626


In [18]:
# Function to have a friendly way to print the events
def eventToString(event):
    return {
        "Problem": lambda x: "(P "+str(event['ProblemID'])+")",
        "Video": lambda x: "(V "+str(event['VideoID'])+")",
        "Forum": lambda x: "(F)"
    }[event['EventType']](event)

In [19]:
TEST_STUDENTS={
    '6ea6949ca133acede360d3573f9d1168b3d70b51':'very good student',
    '3b305429f93de02637949578a5f9e23f13eb0726':'did two problems',
    '865981d5b40a693bafbadae4b1df769be03a25c3':'watched 8 videos',
    '67cdae6073d1089b695e2a615a01187586ad7ba6':'normal student',
    'fb4c81b3df430d1f0fbb8d0ca3e470ac6bf92a2f':'the very best student',
    '295b3496278626b6d337812b1882b756336fd633':'whatdup with this guy?'
}

In [20]:
# Looks at the data for a few students
for studentID,events in rdd_events_by_students.filter(lambda x: x[0] in TEST_STUDENTS.keys()).collect():
    print('\n',studentID,':',TEST_STUDENTS[studentID])
    for event in events:
        print(eventToString(event), end=" ")
    print()


 6ea6949ca133acede360d3573f9d1168b3d70b51 : very good student
(V 8) (V 3) (V 3) (V 4) (V 5) (V 5) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 7) (V 7) (V 7) (V 33) (V 33) (V 33) (P 7) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 39) (V 39) (V 39) (V 41) (V 41) (V 43) (V 47) (V 49) (P 6) (P 6) (P 6) (V 51) (V 51) (V 51) (V 53) (V 53) (V 53) (V 75) (V 75) (V 75) (V 75) (P 12) (V 79) (V 77) (V 81) (V 83) (V 85) (V 87) (V 71) (P 14) (P 14) (V 73) (V 89) (V 91) (V 93) (V 95) (V 95) (V 97) (V 101) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) (V 103) (V 105) (V 107) (V 109) (V 111) (V 113) (V 115) (F) (F) (F) (F) (F) (F) (F) (F) (P 17) (V 109) (V 111) (V 113) (V 107) (V 117) (V 119) (V 121) (V 123) (V 125) (V 127) (V 125) (F) (F) (P 20) (F) (F) (V 125) (V 125) (P 20) (F) 

 865981d5b40a693bafbadae4b1df769be03a25c3 : watched 8 videos
(V 8) (V 3) (V 4) (V 5) (V 6) (V 7) (V 7) (V 33) (V 35) (V 35) (

### Extracting learning patterns
The goal is to extract what a students does between a failed attempt at an assignment and a successful attempt

In [21]:
# The goal is to extract the list corresponding to each Problem
def extractPatterns(events):
    patterns = {}
    for problemID in LECTURES_PER_PROBLEM.keys():
        patterns[problemID]=[]
    # for each problem measure first and last entry
    for event in events:
        if event['EventType']=='Problem':
            patterns[event['ProblemID']].append(event)
        if event['EventType']=='Video':
            if event['VideoID'] in PROBLEM_PER_LECTURE.keys():
                patterns[PROBLEM_PER_LECTURE[event['VideoID']]].append(event)
    return patterns
    

In [22]:
# The goal is to concatenate small sub events into one big event
# For example Play/Pause/Play/Pause/Play/Pause on a video counts as only a Video event
def eventConcat(events):
    if len(events)<2:
        return events
    res=[events[0]]
    for event in events[1:]:
        if (
            event['EventType']!=res[-1]['EventType'] 
            or event.get('VideoID',None)!=res[-1].get('VideoID',None)
            or event['EventType']=='Problem'
        ):
            res.append(event)
    return res

def patternsConcat(patterns):
    res = {}
    for pattern in patterns.keys():
        res[pattern]=eventConcat(patterns[pattern])
    return res

In [23]:
rdd_events_by_students_extract = (rdd_events_by_students
    # x = (studentID,eventList)
    .map(lambda x: (x[0],extractPatterns(x[1])))
    # x = (studentID,{problemID:pattern,...})
    .map(lambda x: (x[0],patternsConcat(x[1])))
)

rdd_events_by_students_extract.persist()

# Looks at the data for a student, now that it's sorted by TimeStamps
def displayStudentsPatterns(rdd,students):
    for studentID,patterns in rdd.filter(lambda x:x[0] in students.keys()).collect():
        print(studentID, students[studentID])
        for pb in sorted(patterns.keys()):
            print(pb,end='\t>>>  ')
            for event in patterns[pb]:
                print(eventToString(event), end=" ")
            print()
        print()

displayStudentsPatterns(rdd_events_by_students_extract,TEST_STUDENTS)

6ea6949ca133acede360d3573f9d1168b3d70b51 very good student
6	>>>  (V 35) (V 37) (V 39) (V 41) (V 43) (V 47) (V 49) (P 6) (P 6) (P 6) 
7	>>>  (V 8) (V 3) (V 4) (V 5) (V 6) (V 7) (V 33) (P 7) 
12	>>>  (V 51) (V 53) (V 75) (P 12) 
14	>>>  (V 79) (V 77) (V 81) (V 85) (V 87) (V 71) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) 
17	>>>  (V 103) (V 105) (V 107) (V 109) (V 111) (V 113) (V 115) (P 17) (V 109) (V 111) (V 113) (V 107) 
20	>>>  (V 117) (V 119) (V 121) (V 123) (V 125) (V 127) (V 125) (P 20) (V 125) (P 20) 

865981d5b40a693bafbadae4b1df769be03a25c3 watched 8 videos
6	>>>  (V 35) 
7	>>>  (V 8) (V 3) (V 4) (V 5) (V 6) (V 7) (V 33) 
12	>>>  
14	>>>  
17	>>>  
20	>>>  

3b305429f93de02637949578a5f9e23f13eb0726 did two problems
6	>>>  (P 6) 
7	>>>  (P 7) 
12	>>>  
14	>>>  
17	>>>  
20	>>>  

fb4c81b3df430d1f0fbb8d0ca3e470ac6bf92a2f the very best student
6	>>>  (V 35) (V 37) (V 39) (V 41) (V 43) (V 47) (V 49) (P 6) (P 6) 
7	>>>  (V 8) (V 3) (V 4) (V 5) (V 4) (V 6) (V 7) (

In [24]:
def eventToString(event,verbosity):
    if verbosity == 'minimal':
        out = {
            "Problem": lambda x:"P",
            "Video": lambda x:"V",
            "Forum": lambda x:"F",
        }
    if verbosity == 'normal':
        out = {
            "Problem": lambda x:"P"+str(x['ProblemID']),
            "Video": lambda x:"V"+str(x['VideoID']),
            "Forum": lambda x:"F",
        }
    return out[event['EventType']](event)

def patternToString(pattern,verbosity):
    link = "" if verbosity == 'minimal' else "-"
    return link.join([eventToString(event,verbosity) for event in pattern])

def containsProblem(pattern):
    return 'P' in patternToString(pattern,'minimal')

def getBeforeLastProblem(pattern):
    indexOfLastProblem = patternToString(pattern,'minimal').rfind('P')
    return pattern[:(indexOfLastProblem+1)]

def getBeforeFirstProblem(pattern):
    indexOfFirstProblem = patternToString(pattern,'minimal').find('P')
    return pattern[:indexOfFirstProblem]

def getAfterFirstProblem(pattern):
    indexOfFirstProblem = patternToString(pattern,'minimal').find('P')
    return pattern[indexOfFirstProblem:]

# Uses the method flatMap on the students patterns table to have a table of all the patterns
rdd_patterns = (rdd_events_by_students_extract
    # x = (studentID,{problemID:pattern,...})
    .flatMap(lambda x: [
        (pb,x[1][pb]) for pb in x[1].keys() 
    ])
    # x = (problemID,pattern)
    .filter(lambda x: containsProblem(x[1]))
    # x = (problemID,pattern)
    .map(lambda x: (x[0],getBeforeLastProblem(x[1])))
)

rdd_patterns.persist()
for pattern in rdd_patterns.map(lambda x: patternToString(x[1],'minimal')).take(10):
    print (pattern)

VVVVVVVVVVVP
VVVVVVVP
VVVP
VVVVP
VVVVPP
PP
P
VVVVP
VVVVVVVP
VVVVVVVP


In [25]:
def getFeatures(problem, pattern):
    features = {}
    
    videoBeforeFirstProblem = [video['VideoID'] for video in getBeforeFirstProblem(pattern)]
    value = 0. if False in [(videoID in videoBeforeFirstProblem) for videoID in LECTURES_PER_PROBLEM[problem]] else 1. 
    features['watchedAllVideosBeforeFirstProblem'] = value

    value = 1. if len(videoBeforeFirstProblem) != len(set(videoBeforeFirstProblem)) else 0.
    features['repeatedVideoBeforeFirstProblem'] = value

    value = sum([event['EventType']=='Video' for event in pattern])
    features['totalNumberOfVideoWatched'] = value

    value = sum([event['EventType']=='Problem' for event in pattern])
    features['totalNumberOfProblemSubmissions'] = value 
            
    value = 1. if [v for v in getAfterFirstProblem(pattern) if v['EventType']=='Video'] else 0.
    features['watchedSomeVideoAfterFirstProblem'] = value

    grades = [problem['Grade'] for problem in pattern if problem['EventType']=='Problem']
    value = 1. if grades[-1] > grades[0] else 0.
    features['increaseGradeFromFirstToLastProblem'] = value

    value = sum([event['EventType']=='Problem' for event in pattern])/len(pattern)
    features['proportionOfProblemEvents'] = value
    
    value = 0.
    features['timeBetweenStartAndFirstProblem'] = value
    
    value = 0.
    features['timeBetweenStartAndLastProblem'] = value
    
    value = 0.
    features['timeBetweenFirstAndLastProblem'] = value
    
    value = grades[0]
    features['firstProblemGrade'] = value
    
    value = grades[-1]
    features['lastProblemGrade'] = value
    
    value = 0.
    features['percentageImprovedBetweenFirstAndLastProblem'] = value
        
    return(features)


FEATURES = [
    'watchedAllVideosBeforeFirstProblem',
    'repeatedVideoBeforeFirstProblem',
    'totalNumberOfVideoWatched',
    'totalNumberOfProblemSubmissions',
    'watchedSomeVideoAfterFirstProblem',
    'increaseGradeFromFirstToLastProblem',
    'proportionOfProblemEvents',
    'timeBetweenStartAndFirstProblem',
    'firstProblemGrade',
    'lastProblemGrade',
    'percentageImprovedBetweenFirstAndLastProblem'
]

rdd_pattern_features = (rdd_patterns
    # x = (problemID, pattern)
    .map(lambda x: (
        x[0],
        patternToString(x[1],'normal'),
        getFeatures(x[0], x[1])
    ))
)

rdd_pattern_features.persist()
for featureVector in rdd_pattern_features.take(5):
    print('problem', featureVector[0])
    print('pattern', featureVector[1])
    pp.pprint(featureVector[2])
    print()

problem 6
pattern V35-V37-V39-V41-V43-V35-V41-V43-V47-V49-V37-P6
{   'firstProblemGrade': 10.0,
    'increaseGradeFromFirstToLastProblem': 0.0,
    'lastProblemGrade': 10.0,
    'percentageImprovedBetweenFirstAndLastProblem': 0.0,
    'proportionOfProblemEvents': 0.08333333333333333,
    'repeatedVideoBeforeFirstProblem': 1.0,
    'timeBetweenFirstAndLastProblem': 0.0,
    'timeBetweenStartAndFirstProblem': 0.0,
    'timeBetweenStartAndLastProblem': 0.0,
    'totalNumberOfProblemSubmissions': 1,
    'totalNumberOfVideoWatched': 11,
    'watchedAllVideosBeforeFirstProblem': 1.0,
    'watchedSomeVideoAfterFirstProblem': 0.0}

problem 7
pattern V8-V3-V4-V5-V6-V7-V33-P7
{   'firstProblemGrade': 10.0,
    'increaseGradeFromFirstToLastProblem': 0.0,
    'lastProblemGrade': 10.0,
    'percentageImprovedBetweenFirstAndLastProblem': 0.0,
    'proportionOfProblemEvents': 0.125,
    'repeatedVideoBeforeFirstProblem': 0.0,
    'timeBetweenFirstAndLastProblem': 0.0,
    'timeBetweenStartAndFirstPro